In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

RAW_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/train/downsampled-1024/1-images/"
VESSELS_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/train/downsampled-1024/2-vessels/"
MASK_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/train/downsampled-1024/mask/"

raw_files = {f[:-4] for f in os.listdir(RAW_PATH) if f.endswith('.jpg')}
seg_files = {f[:-4] for f in os.listdir(RAW_PATH) if f.endswith('.png')}
msk_files = {f[:-4] for f in os.listdir(MASK_PATH) if f.endswith('.png')}

files = list(raw_files.union(seg_files).union(msk_files))[:8]
raw_paths = [RAW_PATH+f+'.jpg' for f in files]
seg_paths = [VESSELS_PATH+f+'.png' for f in files]
mask_paths = [MASK_PATH+f+'.png' for f in files]

In [3]:
files

['504_N', '434_G', '365_G', '101_A', '449_G', '266_D', '420_G', '385_G']

In [4]:
import cv2
import torch
import numpy as np
from steered_cnn.utils.torch import crop_pad 
from jppype import imshow

def load_imgs(path, binarize=False):
    img = np.stack([cv2.imread(p) for p in path]).transpose((0,3,1,2))
    img = img.astype(np.float32)/255
    img = torch.from_numpy(img)
    return img.mean(axis=1)>.5 if binarize else img

raw = load_imgs(raw_paths)
true_vessels = load_imgs(seg_paths, binarize=True)
masks = load_imgs(mask_paths, binarize=True)

In [5]:
from fundus_vessels_toolkit.models import segmentation_model
nnet = segmentation_model().cuda()
pred_vessels = nnet(raw[:4].cuda())

In [6]:
from fundus_vessels_toolkit.metrics import CfgSumClDiceF1Topo

In [7]:
cfgMetrics = CfgSumClDiceF1Topo(data={'max_struct_width': 12})
metric = cfgMetrics.create()

In [11]:
metric(*cfgMetrics.prepare_data(pred_vessels[:4,1]>.5*masks[:4].cuda(), true_vessels[:4].cuda()*masks[:4].cuda()))

tensor(0.8097)

In [ ]:
v = imshow(raw[0]*255)
v.add_label(pred_vessels[0][1]>.5)
v